In [1]:
import h5py
import os
import pathlib
import twixtools

DIR = os.getcwd()
print('DIR: ', DIR)

DIR:  /home/zhengguo/Softwares/SEMAC


In [2]:
datastr = DIR + '/data/meas_MID00321_FID02913_pd_tse_semac_cor_full_12SEMAC.dat'

outprefstr = datastr.split('.dat')[0]
pathlib.Path(outprefstr).mkdir(parents=True, exist_ok=True)

twixobj = twixtools.read_twix(datastr)

twix = twixobj[-1]

hdr_twix = twixobj[-1]['hdr']
MB = hdr_twix['Phoenix']['sSliceAcceleration']['lMultiBandFactor']
N_slices = hdr_twix['Phoenix']['sSliceArray']['lSize']
N_segments = hdr_twix['Phoenix']['sFastImaging']['lSegments']
N_EchoTrainLength = int(hdr_twix['Meas']['EchoTrainLength'])
N_Accel_PE = hdr_twix['Phoenix']['sPat']['lAccelFactPE']

print('> multi-band ' + str(MB) +\
      ', slices ' + str(N_slices) +\
      ', segments ' + str(N_segments) +\
      ', echo train length ' + str(N_EchoTrainLength) +\
      ', iPat ' + str(N_Accel_PE))

Software version: VD/VE (!?)

Scan  0


100%|██████████| 23.6M/23.6M [00:00<00:00, 264MB/s]


WARNING more than one slice. Taking first one..
Scan  1


100%|██████████| 4.86G/4.86G [00:07<00:00, 706MB/s]

> multi-band 1, slices 26, segments 1, echo train length 13, iPat 1


In [6]:
REMOVE_OS = True

mapped = twixtools.map_twix(datastr)

# kdat twix
kdat_twix = mapped[-1]['image']

kdat_twix.flags['regrid'] = True
kdat_twix.flags['remove_os'] = REMOVE_OS
kdat_twix.flags['zf_missing_lines'] = True
kdat_twix.flags['average']['Seg'] = True  # take together all echo trains

Software version: VD/VE (!?)

Scan  0


100%|██████████| 23.6M/23.6M [00:00<00:00, 253MB/s]


WARNING more than one slice. Taking first one..
Scan  1


100%|██████████| 4.86G/4.86G [00:07<00:00, 741MB/s] 


In [7]:
print(kdat_twix.shape)
print(kdat_twix.flags)

N_semac = kdat_twix.shape[-6]  # Par
N_slice = kdat_twix.shape[-5]  # Sli
N_y = kdat_twix.shape[-3]      # Lin
N_x = kdat_twix.shape[-1]      # Col
N_coil = kdat_twix.shape[-2]   # Cha
N_echo= kdat_twix.shape[-11]   # Seg

(1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 12, 26, 1, 299, 12, 288)
{'average': {'Ide': True, 'Idd': True, 'Idc': True, 'Idb': True, 'Ida': True, 'Seg': True, 'Set': False, 'Rep': False, 'Phs': False, 'Eco': False, 'Par': False, 'Sli': False, 'Ave': True, 'Lin': False, 'Cha': False, 'Col': False}, 'remove_os': True, 'regrid': True, 'apply_fftscale': False, 'apply_rawdatacorr': False, 'squeeze_ave_dims': False, 'squeeze_singletons': False, 'skip_empty_lead': False, 'zf_missing_lines': True, 'user_func': []}


In [5]:
for s in range(N_slice):

    k = kdat_twix[..., [s], :, :, :, :]

    f = h5py.File(outprefstr + '/semac_slice_' + str(s).zfill(3) + '.h5', 'w')
    f.create_dataset('kspace', data=k)
    f.close()